In [2]:
#Package Installation 
!pip install pandas
!pip install mysql-connector-python


In [3]:
#Import Packages
import pandas as pd
import mysql.connector as db 

In [4]:
#Creating Dataframes and read CSV files
df1 = pd.read_csv("customers.csv")
df2 = pd.read_csv("customer_reviews.csv")
df3 = pd.read_csv("customer_journey.csv")
df4 = pd.read_csv("products.csv")
df5 = pd.read_csv("engagement_data.csv")
df6 = pd.read_csv("geography.csv")

In [5]:
#Check duplicates all the dataframes

# df1.duplicated().sum()
# df2.duplicated().sum()
# df3.duplicated().sum()
#df4.duplicated().sum()
# df5.duplicated().sum()
# df6.duplicated().sum()

In [11]:
# Convert datatype if needed
df2['ReviewDate'] = pd.to_datetime(df2.ReviewDate)
df3['VisitDate'] = pd.to_datetime(df3.VisitDate)
df3['Duration'] = df3['Duration'].fillna(0).astype(int) 
df5['EngagementDate'] = pd.to_datetime(df5.EngagementDate)
df5['Views'] = df5['Views'].astype(int)
df5['Clicks'] = df5['Clicks'].astype(int)


In [6]:
#Check for null values
df3 = pd.read_csv("customer_journey.csv")
df3.isnull().sum()

JourneyID      0
CustomerID     0
ProductID      0
VisitDate      0
Stage          0
Action         0
Duration      14
dtype: int64

In [7]:
#check for null values
df3 = pd.read_csv("customer_journey.csv")
df3.isnull()


,JourneyID,CustomerID,ProductID,VisitDate,Stage,Action,Duration
0,False,False,False,False,False,False,True
1,False,False,False,False,False,False,True
2,False,False,False,False,False,False,False
3,False,False,False,False,False,False,True
4,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...
95,False,False,False,False,False,False,False
96,False,False,False,False,False,False,False
97,False,False,False,False,False,False,False
98,False,False,False,False,False,False,False


In [8]:
#Drop null values 
df3.dropna(inplace=True)
df3.rename(columns={'Action':'Action_by_customers'},inplace=True)
df3 = df3.reset_index(drop=True)


In [9]:
# Read 
df5 = pd.read_csv("engagement_data.csv")
df5.isnull().sum()

EngagementID           0
ContentID              0
ContentType            0
Likes                  0
EngagementDate         0
CampaignID             0
ProductID              0
ViewsClicksCombined    0
dtype: int64

In [10]:
#Split combined columns
df5[['Views', 'Clicks']] = df5['ViewsClicksCombined'].str.split('-', expand = True)
df5.drop(columns=['ViewsClicksCombined'], inplace = True)
df5

,EngagementID,ContentID,ContentType,Likes,EngagementDate,CampaignID,ProductID,Views,Clicks
0,1,39,Blog,190,2023-08-30,1,9,1883,671
1,2,48,Blog,114,2023-03-28,18,20,5280,532
2,3,16,video,32,2023-12-08,7,14,1905,204
3,4,43,Video,17,2025-01-21,19,20,2766,257
4,5,16,newsletter,306,2024-02-21,6,15,5116,1524
...,...,...,...,...,...,...,...,...,...
95,96,17,Blog,32,2024-07-15,14,13,712,142
96,97,34,blog,3,2025-07-01,16,7,696,47
97,98,44,blog,0,2025-08-20,11,8,231,10
98,99,26,Blog,119,2024-04-15,17,4,3511,479


In [12]:
#Setup connection
user  = 'newuser'
passcode = "Apps@5566"
host = 'localhost'
db_name = "shopeasy"

In [13]:
#db connection
db_connection = db.connect(

                host  = host,
                user = user,
                password = passcode,
                database = db_name
)

In [14]:
db_connection

In [15]:
#Create Cursor for db_connection
cursor = db_connection.cursor()

#Insert into Customer table
insert_query = """
INSERT INTO customers (CustomerID, CustomerName, Email, Gender, Age, GeographyID)
VALUES (%s, %s, %s, %s, %s, %s) """

#Convert df1 to list 
cursor.executemany(insert_query, df1.values.tolist())
db_connection.commit()

In [16]:
#Insert into Customer_reviews table
insert_query = """
INSERT INTO customer_reviews (ReviewID, CustomerID, ProductID, ReviewDate, Rating, ReviewText)
VALUES (%s, %s, %s, %s, %s, %s)
"""
#Convert df2 to list 
cursor.executemany(insert_query, df2.values.tolist())  
db_connection.commit()

In [17]:
#Insert into Customer table_journey
insert_query = """
INSERT INTO customer_journey (JourneyID, CustomerID, ProductID, VisitDate, Stage, Action_by_customers, Duration)
VALUES (%s, %s, %s, %s, %s, %s, %s)
"""
#Convert df3 to list 
cursor.executemany(insert_query, df3.values.tolist())  
db_connection.commit()

In [18]:
#Insert into Products table
insert_query = """
INSERT INTO products (ProductID, ProductName, Category, Price)
VALUES (%s, %s, %s, %s)
"""
#Convert df4 to list 
cursor.executemany(insert_query, df4.values.tolist())  
db_connection.commit()

In [19]:
df5

,EngagementID,ContentID,ContentType,Likes,EngagementDate,CampaignID,ProductID,Views,Clicks
0,1,39,Blog,190,2023-08-30,1,9,1883,671
1,2,48,Blog,114,2023-03-28,18,20,5280,532
2,3,16,video,32,2023-12-08,7,14,1905,204
3,4,43,Video,17,2025-01-21,19,20,2766,257
4,5,16,newsletter,306,2024-02-21,6,15,5116,1524
...,...,...,...,...,...,...,...,...,...
95,96,17,Blog,32,2024-07-15,14,13,712,142
96,97,34,blog,3,2025-07-01,16,7,696,47
97,98,44,blog,0,2025-08-20,11,8,231,10
98,99,26,Blog,119,2024-04-15,17,4,3511,479


In [20]:
#Insert into Engagement_data table
insert_query = """
INSERT INTO engagement_data (EngagementID, ContentID, ContentType, Likes, EngagementDate, CampaignID, ProductID, Views, Clicks)
VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)
"""
#Convert df5 to list 

cursor.executemany(insert_query, df5.values.tolist())  
db_connection.commit()

In [21]:
#Insert into Geography table
insert_query = """
INSERT INTO geography (GeographyID, Country, City)
VALUES (%s, %s, %s)
"""

#Convert df6 to list 
cursor.executemany(insert_query, df6.values.tolist())  
db_connection.commit()


In [23]:
#Close connection
db_connection.close()